In [ ]:
!python3 -m pip install ultralytics opencv-python ipywidgets

In [ ]:
!python --version

In [ ]:
!pip install jtop

In [ ]:
from jtop import jtop

with jtop() as jetson:
    # jetson.ok() will provide the proper update frequency
    while jetson.ok():
        # Read tegra stats
        print(jetson.stats)

In [ ]:
import cv2
import torch
from ultralytics import YOLO
from IPython.display import display, Image
import ipywidgets as widgets
import threading
import time
from matplotlib import rcParams

# Load the YOLOv10nmodel
model = YOLO('yolov10s.pt') #load
model.to('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize the video capture
cap = cv2.VideoCapture('/dev/video0')

# Create an output widget
output_widget = widgets.Image()
display(output_widget)

def process_frame(frame):
    results = model(frame)
    annotated_frame = results[0].plot()
    return cv2.cvtColor(annotated_frame, cv2.COLOR_BGR2RGB)

def video_feed():
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        processed_frame = process_frame(frame)
        _, jpeg = cv2.imencode('.jpg', processed_frame)
        output_widget.value = jpeg.tobytes()
        time.sleep(0.1)  # Adjust this value to control frame rate

# Start the video feed in a separate thread
thread = threading.Thread(target=video_feed)
thread.daemon = True
thread.start()

# Keep the notebook running
widgets.interactive(lambda x: x, x=widgets.IntSlider(min=1, max=10, step=1, value=5))

In [ ]:
!pip install gradio

In [ ]:
import cv2
import gradio as gr
from ultralytics import YOLO

# Load the YOLOv10 model
model = YOLO('yolov10s.pt')

def process_video():
    cap = cv2.VideoCapture('/dev/video0')
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Perform detection
        results = model(frame)
        
        # Draw bounding boxes
        annotated_frame = results[0].plot()
        
        yield annotated_frame

    cap.release()


# Create Gradio interface
iface = gr.Interface(
    fn=process_video,
    inputs=None,
    outputs="image",
    live=True
)

# Launch the interface
iface.launch(server_name="192.168.1.238", server_port=7866, share=True)

In [ ]:
!nvgstcapture-1.0 --automate --capture-auto


In [ ]:
!pip3 install jetson-examples

In [ ]:
!v4l2-ctl --list-devices

In [ ]:
!v4l2-ctl --list-formats-ext

In [ ]:
import cv2
from ultralytics import YOLO

# Load the YOLOv8 model (nano version for lightweight performance)
model = YOLO('yolov8n.pt')

# GStreamer pipeline to access the IMX219 camera at 1280x720 resolution and 60 fps
gst_pipeline = (
    "nvarguscamerasrc ! "
    "video/x-raw(memory:NVMM), width=1280, height=720, framerate=60/1 ! "
    "nvvidconv ! video/x-raw, format=BGRx ! "
    "videoconvert ! appsink"
)

# Open the camera using the GStreamer pipeline
cap = cv2.VideoCapture(gst_pipeline, cv2.CAP_GSTREAMER)

# Check if the camera opened successfully
if not cap.isOpened():
    print("Error: Could not open video stream from the camera.")
    exit()

# Process the video stream
while True:
    ret, frame = cap.read()

    # Check if frame was successfully captured
    if not ret:
        print("Error: Failed to grab frame from the camera.")
        break
    
    # Perform YOLOv8 detection on the current frame
    results = model(frame)
    
    # Annotate the frame with bounding boxes
    annotated_frame = results[0].plot()

    # Display the frame with YOLOv8 annotations
    cv2.imshow('YOLOv8 Object Detection', annotated_frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close OpenCV windows
cap.release()
cv2.destroyAllWindows()
